# Process BAG

(should work on Mac or Windows too, if duckdb installed)

the original problem is that BAG is published as 2000+ xml files (zipped) and QGIS could more easily open a FlatGeobuf or a set of (Geo)parquet files. In theory QGIS could convert but it's hard to open all these at once, so `duckdb` can be used.

In [ ]:
# Download can take 40+(!) minutes, this is due to the the uplink speed from
# Kadaster though.
!curl -L -o lvbag-extract-nl.zip "https://service.pdok.nl/kadaster/adressen/atom/v1_0/downloads/lvbag-extract-nl.zip"

In [ ]:
import zipfile

LOCAL_FILE_PATH = "./lvbag-extract-nl.zip"

with zipfile.ZipFile(LOCAL_FILE_PATH, "r") as zip_ref:
    zip_ref.extractall("./lvbag-extract-nl/")

In [ ]:
import glob

pnd_file = glob.glob("./lvbag-extract-nl/9999PND*.zip")[0]
pnd_file

## Option 1: DuckDB

In [ ]:
import duckdb

duckdb.sql("""install spatial; load spatial""")

In [ ]:
# 40 min
duckdb.sql(
    f"""COPY (
    SELECT
        *
    FROM st_read('/vsizip/{pnd_file}')
) TO 'bag.fgb' (
    FORMAT GDAL,
    DRIVER flatgeobuf
)
"""
)

This flatgeobuf file, while large, can actually be efficiently opened in QGIS after download.

## Option 2: ogr2ogr (same speed though)

In [ ]:
PND_PATH = "./lvbag-extract-nl/9999PND/"

with zipfile.ZipFile(pnd_file, "r") as zip_ref:
    zip_ref.extractall(PND_PATH)

In [ ]:
!apt-get install -y gdal-bin

In [ ]:
# 36 min
!ogr2ogr ./bag_ogr.fgb {PND_PATH}